In [3]:
import pandas as pd
import numpy as np
import pickle
import os
import glob
from datetime import datetime as dt
%matplotlib inline

from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

## for all csv's in the SF folder, concatenate into one dataframe

In [53]:
files = glob.glob('SF/*.gz')


In [54]:
df_sf = pd.concat([pd.read_csv(fp, parse_dates = ['calendar_last_scraped','last_scraped', 'first_review', 'last_review', 'host_since']) for fp in files], ignore_index=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


KeyboardInterrupt: 

In [9]:
len(df_sf)

204559

## Drop the rows where the number of reviews is greater than 0 and the reviews per month is zero

In [40]:
df_sf = df_sf[(df_sf['reviews_per_month'].notnull()) | ((df_sf['number_of_reviews']== 0) & (df_sf['reviews_per_month'].isnull()))]

## fill 0 in the reviews per month where total reviews = 0

In [41]:
df_sf.reviews_per_month.fillna(value=0, inplace=True)

## drop listings where price is 0

In [42]:
df_sf['price'].replace('[\$,]', '', regex=True, inplace=True)
df_sf['price'] = df_sf.price.astype(float) ##
df_sf.drop(df_sf[df_sf.price == 0].index, inplace=True)
df_sf.drop(df_sf[df_sf.price > 20000].index, inplace=True)

In [13]:
len(df_sf)

204233

In [43]:
df_sf.cleaning_fee.fillna(value='$0.00', inplace=True)
df_sf['cleaning_fee'].replace('[\$,]', '', regex=True, inplace=True)
df_sf['cleaning_fee'] = df_sf.cleaning_fee.astype(float) ##

In [44]:
df_sf['price_actual'] = df_sf['price'].add(df_sf['cleaning_fee'], fill_value=0)

In [16]:
# df_sf[df_sf['cleaning_fee'].notnull()][['price_new', 'price', 'cleaning_fee']].head(20)

## create year and month columns

In [56]:
df_sf['year'] = df_sf.calendar_last_scraped.apply(lambda x: x.year)
df_sf['month'] = df_sf.calendar_last_scraped.apply(lambda x: x.month)

In [18]:
len(df_sf)

204233

In [78]:
# df_sf_2017[df_sf_2017['review_scores_rating'].notnull()]['review_scores_rating'].mean() MEAN IS 94.76

## Asking for at least 3 months minimum is not a realistic listing

In [46]:
df_sf.drop(df_sf[df_sf.minimum_nights>90].index, inplace=True)

## Fill nulls

In [47]:
df_sf.cancellation_policy.fillna(value='none', inplace=True)

#those who aren't rated are grouped together
df_sf.review_scores_rating.fillna(value=0, inplace=True)

#do host fill out information or not
df_sf.loc[df_sf.host_about.notnull(), 'host_about_filled'] = True
df_sf.loc[df_sf.host_about.isnull(), 'host_about_filled'] = False
df_sf.loc[df_sf.host_picture_url.notnull(), 'host_picture_url_filled'] = True
df_sf.loc[df_sf.host_picture_url.isnull(), 'host_picture_url_filled'] = False
df_sf.loc[df_sf.access.notnull(), 'access_filled'] = True
df_sf.loc[df_sf.access.isnull(), 'access_filled'] = False
df_sf.loc[df_sf.house_rules.notnull(), 'house_rules_filled'] = True
df_sf.loc[df_sf.house_rules.isnull(), 'house_rules_filled'] = False
df_sf.loc[df_sf.space.notnull(), 'space_filled'] = True
df_sf.loc[df_sf.space.isnull(), 'space_filled'] = False

df_sf['extra_people'].replace('[\$,]', '', regex=True, inplace=True)

df_sf.property_type.fillna(value = 'Other', inplace=True) ##

#bedroom average is 1. 83 nulls. 
df_sf.bedrooms.fillna(value = 1, inplace=True) ##
df_sf.bedrooms.replace(0, 1, inplace=True) ##

df_sf.beds.fillna(value = 1, inplace=True) ##

df_sf['log_price'] = df_sf['price_actual'].apply(np.log)

#value of the place you are booking. take log to get 
df_sf['price_per_bedroom'] = df_sf['log_price']/df_sf['bedrooms'] ##

#how much each person pays
df_sf['price_per_guest'] = df_sf['log_price']/(df_sf['guests_included']+1) ##adding one to included booker

#do you get your own bed?
df_sf['guest_per_bed'] = df_sf['guests_included']/df_sf['beds'] ##

#do you get your own bedroom?
df_sf['guest_per_bedroom'] = df_sf['guests_included']/df_sf['bedrooms'] ##

In [81]:
df_sf.monthly_price.value_counts().sort_index

<bound method Series.sort_index of $3,000.00     1659
$4,000.00     1647
$2,500.00     1601
$3,500.00     1554
$5,000.00     1448
$4,500.00     1371
$6,000.00      951
$2,000.00      863
$1,800.00      760
$1,500.00      725
$3,200.00      725
$2,800.00      671
$5,500.00      657
$3,600.00      641
$3,800.00      567
$8,000.00      507
$2,400.00      489
$2,200.00      479
$6,500.00      459
$7,000.00      435
$4,200.00      433
$10,000.00     405
$2,600.00      392
$3,300.00      382
$1,300.00      366
$1,700.00      361
$1,400.00      360
$4,800.00      332
$1,900.00      332
$1,600.00      331
$1,650.00      329
$2,100.00      314
$7,500.00      297
$2,300.00      294
$2,900.00      284
$3,900.00      279
$3,400.00      268
$3,700.00      266
$4,250.00      259
$1,100.00      248
$5,400.00      248
$4,300.00      241
$2,750.00      239
$8,500.00      239
$5,600.00      231
$4,900.00      221
$2,700.00      220
$2,350.00      218
$1,450.00      211
$2,250.00      204
$9,000.00      

In [48]:
df_sf['host_since_year'] = df_sf.host_since.apply(lambda x: x.year)
df_sf['host_since_year'].fillna(value=2017, inplace=True)
df_sf['host_length'] = 2017 -df_sf['host_since_year']

In [22]:
df_sf[df_sf['review_scores_rating'].isnull()][['review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', 'review_scores_communication','review_scores_location', 'review_scores_rating' , 'review_scores_value', 'reviews_per_month']].head(300)





,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month


In [64]:
# df_sf_2017[df_sf_2017['host_response_rate'].notnull()]['host_response_rate'].

In [26]:
#find the mean response rate

x = df_sf_2017[df_sf_2017['host_response_rate'].notnull()]['host_response_rate']
x.replace('[\%,]', '', regex=True, inplace=True)
x.astype('int64',errors='ignore')
mean_response_rate = x.real.astype('int64').mean()
mean_response_rate

95.90738384554297

In [49]:
df_sf.host_response_rate.fillna(value='96%', inplace=True) ##96% is the mean
df_sf['host_response_rate'].replace('[\%,]', '', regex=True, inplace=True)
df_sf['host_response_rate'] = df_sf.host_response_rate.astype(int) ##

## 2017 dataframe

In [ ]:
df_sf_2017 = df_sf[df_sf['year']==2017]
df_sf_2017.loc[df_sf_2017.cancellation_policy == 'flexible_new', 'cancellation_policy'] = "flexible"

## 2.57 is the cutoff reviews per month to be in the top 20% of listings for all of SF in 2017

In [57]:
top_twenty = len(df_sf_2017)*.2
top_twenty

20743.600000000002

In [52]:
df_sf_2017.sort_values(by='reviews_per_month', ascending=False).iloc[20743,:]

access                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      The studio is on the ground floor of our home, we live upstairs. The bottom floor basement/art studio/exercise room leads out to a large backyard. Although the studio is private, and separate from our living area, you’ll have access to the following shared amenities: backyard, deck and sitting area. Washer/Dryer, exercise machine (in basement/art studio). The en

In [70]:
# df_sf.loc[df_sf.reviews_per_month >= 2.54, 'popular'] = True
# df_sf.loc[df_sf.reviews_per_month < 2.54, 'popular'] = False

In [28]:
df_sf_2017.loc[df_sf_2017.reviews_per_month >= 2.66, 'popular'] = True
df_sf_2017.loc[df_sf_2017.reviews_per_month < 2.66, 'popular'] = False

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
notpopular = df_sf_2017[df_sf_2017['popular']==False]

In [7]:
popular = df_sf_2017[df_sf_2017['popular']==True]

In [8]:
def turn_to_set(host_id, list_id):
    newSet = set()
    for i,v in enumerate(host_id):
        newSet.add((host_id[i], list_id[i]))
    return newSet

In [9]:
#create dictionaries that contain the unique listings for each month

def unique_listings_per_month():
    pop_dict = {}
    for i in range(4,13):
        host_id = popular[(popular['month']==i)]['host_id']
        list_id = popular[(popular['month']==i)]['id']

        host_id = np.array(host_id)
        list_id = np.array(list_id)

        mySet = turn_to_set(host_id, list_id)

        pop_dict["month_" + str(i)] = mySet


    notpop_dict = {}
    for i in range(4,13):
        host_id_notpop = notpopular[(notpopular['month']==i)]['host_id']
        list_id_notpop = notpopular[(notpopular['month']==i)]['id']

        host_id_notpop = np.array(host_id_notpop)
        list_id_notpop = np.array(list_id_notpop)

        mySet = turn_to_set(host_id_notpop, list_id_notpop)

        notpop_dict["month_" + str(i) + "_notpop"] = mySet
    
    return pop_dict, notpop_dict


In [58]:
pop_dict, notpop_dict = unique_listings_per_month()

In [63]:
month_to_month_change = np.zeros(shape = (7, 8))
count = 0
for i in range(4,12):
    diff_first_month = pop_dict['month_'+str(i)].difference(pop_dict['month_'+str(i+1)]) #elements popular in first month but not popular in second month. Did they become unpopular or did the listing disappear?
    month_to_month_change[0][count] = len(diff_first_month)
    
    diff_second_month = pop_dict['month_'+str(i+1)].difference(pop_dict['month_'+str(i)]) #elements that were popular in second month but not popular in first month. Were they an unpopular listing in the first month?
    month_to_month_change[1][count] = len(diff_second_month) 

    month_to_month_change[2][count] = len(pop_dict['month_'+str(i)].intersection(pop_dict['month_'+str(i+1)])) #continued to be popular in both first and second month

    month_to_month_change[3][count] = len(diff_first_month.intersection(notpop_dict['month_'+str(i+1)+'_notpop'])) #popular elements in first month and not popular in second month. They are no longer popular, but still a listing

    month_to_month_change[4][count] = len(diff_first_month.difference(notpop_dict['month_'+str(i+1)+'_notpop'])) #popular elements in first month and not in second month but because it no longer a listing in second month

    month_to_month_change[5][count] = len(diff_second_month.intersection(notpop_dict['month_'+str(i)+'_notpop'])) #elements that were popular in second month and was unpopular in first month

    month_to_month_change[6][count] = len(diff_second_month.difference(notpop_dict['month_'+str(i)+'_notpop'])) #elements that were popular in second month but not listed in first month
    
    count +=1

month_to_month_change

array([[ 181.,  138.,  149.,  137.,  178.,  239.,  152.,  198.],
       [ 185.,  216.,  193.,  265.,  205.,  247.,  212.,  173.],
       [1342., 1389., 1456., 1512., 1599., 1565., 1660., 1674.],
       [  46.,   66.,   60.,   62.,   78.,   76.,   71.,  108.],
       [ 135.,   72.,   89.,   75.,  100.,  163.,   81.,   90.],
       [ 142.,  141.,  139.,  216.,  179.,   78.,  152.,  118.],
       [  43.,   75.,   54.,   49.,   26.,  169.,   60.,   55.]])

In [61]:
# the total number of popular listings per month

for i in range (4, 13):
    print(i, len(pop_dict['month_' +str(i)]))

4 1523
5 1527
6 1605
7 1649
8 1777
9 1804
10 1812
11 1872
12 1847


In [64]:
#percentage of listings that remain popular

month_to_month_change[2] / (month_to_month_change[0] + month_to_month_change[2])

array([0.88115561, 0.90962672, 0.90716511, 0.91691935, 0.89983118,
       0.86751663, 0.91611479, 0.89423077])

In [65]:
#average percentage of listings that remain popular
(month_to_month_change[2] / (month_to_month_change[0] + month_to_month_change[2])).mean()

0.8990700190547257

In [35]:
#percentage that is no longer a listing in the next month (they dropped out the next month)
np.round((month_to_month_change[4]/month_to_month_change[0])*100,0) 

array([75., 52., 60., 55., 56., 68., 53., 45.])

In [36]:
#average dropout
np.round((month_to_month_change[4]/month_to_month_change[0])*100,0).mean()

58.0

In [37]:
#percentage that were popular in first month but then became unpopular the second month
np.round((month_to_month_change[3]/month_to_month_change[0])*100,0) 

array([25., 48., 40., 45., 44., 32., 47., 55.])

In [38]:
#average that became unpopular in one month
np.round((month_to_month_change[3]/month_to_month_change[0])*100,0).mean()

42.0

In [39]:
#percentage that were not popular in the first month but became popular in the second month
np.round(month_to_month_change[5]/month_to_month_change[1]*100,0) 

array([77., 65., 72., 82., 87., 32., 72., 68.])

In [40]:
#average that became popular in one month
np.round(month_to_month_change[5]/month_to_month_change[1]*100,0).mean()

69.375

## pickle the two dataframes

In [71]:
# df_sf.to_pickle('data_sf.p')
# df_sf_2017.to_pickle('data_sf_2017.p')

## Should instead save to json file instead of pickle flie

In [33]:
df_sf_2017.to_json('df_sf_2017',orient='columns')

In [34]:
!ls -l

total 1390084
-rw-rw-r-- 1 ubuntu ubuntu    164857 Aug  7 19:22 data_cleaning.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 654289300 Aug  7 19:24 df_sf_2017
-rw-rw-r-- 1 ubuntu ubuntu 651191533 Aug  4 01:22 df_sf_2017_old
-rw-rw-r-- 1 ubuntu ubuntu   1036920 Aug  6 23:27 EDA_SF_2017.ipynb
-rw-rw-r-- 1 ubuntu ubuntu   4797091 Aug  2 18:26 EDA_top_20.ipynb
-rw-rw-r-- 1 ubuntu ubuntu    398602 Aug  6 22:46 Ensemble Model Test.ipynb
-rw-rw-r-- 1 ubuntu ubuntu    207836 Aug  7 18:32 nb_model0.p
-rw-rw-r-- 1 ubuntu ubuntu    206492 Aug  7 18:33 nb_model1.p
-rw-rw-r-- 1 ubuntu ubuntu    410364 Aug  1 20:24 nb_model_2.p
-rw-rw-r-- 1 ubuntu ubuntu    208124 Aug  7 18:33 nb_model2.p
-rw-rw-r-- 1 ubuntu ubuntu    336700 Aug  7 17:01 nb_model3.p
-rw-rw-r-- 1 ubuntu ubuntu    335228 Aug  7 17:02 nb_model4.p
-rw-rw-r-- 1 ubuntu ubuntu    338844 Aug  7 17:02 nb_model5.p
-rw-rw-r-- 1 ubuntu ubuntu    334524 Aug  7 17:03 nb_model6.p
-rw-rw-r-- 1 ubuntu ubuntu    339868 Aug  7 17:03 nb_model7.p
-rw-rw-r-- 1 ubuntu 

In [5]:
df_sf_2017 = pd.read_json('df_sf_2017')

In [4]:
df_sf_2017.price_per_bedroom.isnull().sum()

NameError: name 'df_sf_2017' is not defined